# P3. Wrangle Open Street Map Data

## Introduction

<img style="float: center;" src="img/logo.png">


I chose the area of Oslo, the capital of Norway. The snapshot of it can be downloaded from [mapzen](https://s3.amazonaws.com/metro-extracts.mapzen.com/oslo_norway.osm.bz2).

There is a makefile in this folder, which does the download and unpacks the file for the whole dataset:

```bash
$ make download
```

Note that it's a big file (1.3Gb). There is a smaller sample included for testing, `stovner.osm`


In [1]:
# load the ipython-sql extention to run inline SQL and setup syntax coloring for it
%load_ext sql

import IPython
js = "IPython.CodeCell.config_defaults.highlight_modes['magic_sql'] = {'reg':[/^%%sql/]};"
IPython.core.display.display_javascript(js, raw=True)

In [9]:
%sql sqlite:///oslo_norway.db

'Connected: None@oslo_norway.db'

In [10]:
import xml.etree.cElementTree as ET
import pprint

In [11]:
DATA_PATH = 'oslo_norway.osm'
DATA_PATH = 'stovner.osm'


In [12]:
%%time
from collections import defaultdict

node_counts = defaultdict(int)
for event, node in ET.iterparse(DATA_PATH, events=('start',)):
    node_counts[node.tag] += 1
        
node_counts

CPU times: user 376 ms, sys: 12 ms, total: 388 ms
Wall time: 415 ms


In [13]:
node_counts

defaultdict(int,
            {'bounds': 1,
             'member': 16509,
             'nd': 36455,
             'node': 32632,
             'osm': 1,
             'relation': 134,
             'tag': 30977,
             'way': 4837})

In [16]:
"""
%%time
from collections import defaultdict

node_counts = defaultdict(int)
for event, node in ET.iterparse(DATA_PATH, events=('start',)):
    if elem.tag == 'way':
        for tag in elem.iter('tag'):
            if is_street_name(tag):
                audit_street_type(street_types, tag.attrib['v'])
pprint.pprint(dict(street_types))        
"""

"\n%%time\nfrom collections import defaultdict\n\nnode_counts = defaultdict(int)\nfor event, node in ET.iterparse(DATA_PATH, events=('start',)):\n    if elem.tag == 'way':\n        for tag in elem.iter('tag'):\n            if is_street_name(tag):\n                audit_street_type(street_types, tag.attrib['v'])\npprint.pprint(dict(street_types))        \n"

In [25]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

KEY_TYPES = [
    ('problemchars', problemchars),
    ('lower_colon', lower_colon),
    ('lower', lower),
    ('other', re.compile(r'.*'))
]

def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if k is not None:
            for key, pat in KEY_TYPES:
                 if pat.search(k) is not None:
                     keys[key] += 1
                     break
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename, events=('start',)):
        keys = key_type(element, keys)
    return keys

keys = process_map(DATA_PATH)
pprint.pprint(keys)

{'lower': 14075, 'lower_colon': 16764, 'other': 138, 'problemchars': 0}


In [28]:
def get_user(element):
    return


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename, events=('start',)):
        if 'user' in element.attrib:
            users.add(element.attrib['user'])
        
    return users

users = process_map(DATA_PATH)
pprint.pprint(users)

{'673a',
 'ArneH',
 'BRoboMan',
 'BiIbo',
 'Bjørn Larsen',
 'BjørnN',
 'Bobkare',
 'Competizione',
 'EOE',
 'Einar Bøhn',
 'EirikGr',
 'Entur AS | Jan Brekke',
 'Entur AS | Johan Wiklund',
 'Essin',
 'FredrikLindseth',
 'Fudgiepoos',
 'Gazer75',
 'Gazer75_import',
 'Geirkr',
 'Gnonthgol_import',
 'Gustav F',
 'HansHE',
 'HelgeO',
 'Hjart',
 'Håkon',
 'Kai Stian',
 'KonTur',
 'Kurtn',
 'LA2',
 'LilleLudo',
 'NKA',
 'Noen',
 'Pizzabolle',
 'Polarbear',
 'RaBalder',
 'Ramseslegrand',
 'Reitstoen',
 'Rian76',
 'Skywave',
 'T2norway',
 'Thpe',
 'Tompro',
 'Tractor',
 'Tronikon',
 "Turleder'n",
 'Vedeler',
 'WJtW',
 'ZorroIII',
 'abel801',
 'aighes',
 'aytfadc',
 'brunnen',
 'cgu66',
 'dkart',
 'eiriks',
 'gormur',
 'haakonst',
 'hecktor',
 'hofoen',
 'jejacobsen',
 'kao',
 'kerosin',
 'km2bp',
 'knuthaug',
 'kristaga',
 'landfahrer',
 'michalzz',
 'oddbear',
 'opani',
 'oyvind',
 'piligab',
 'pizzaiolo',
 'polarbear42',
 'qwem',
 'rmikke',
 'roemcke',
 'rubund',
 'rubund_import',
 'tibnor',

In [32]:
"""
Your task in this exercise has two steps:

- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "example.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    suffix = name.split(' ')[-1]
    if suffix in mapping:
        return name.replace(suffix, mapping[suffix])
    return name


def test():
    st_types = audit(DATA_PATH)
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print(name, "=>", better_name)
            if name == "West Lexington St.":
                assert better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                assert better_name == "Baldwin Road"
test()

{'Bekkevollveien': {'Bekkevollveien'},
 'Bergtunveien': {'Bergtunveien'},
 'Bikuben': {'Bikuben'},
 'Bjerkeliveien': {'Bjerkeliveien'},
 'Bjørnheimveien': {'Bjørnheimveien'},
 'Brolandsveien': {'Brolandsveien'},
 'Ellingsrudveien': {'Ellingsrudveien'},
 'Enebostubben': {'Enebostubben'},
 'Eneboveien': {'Eneboveien'},
 'Fjellstuveien': {'Fjellstuveien'},
 'Folkvangveien': {'Folkvangveien'},
 'Fredheimveien': {'Fredheimveien'},
 'Granliveien': {'Granliveien'},
 'Hagastubben': {'Hagastubben'},
 'Hagaveien': {'Hagaveien'},
 'Haugenstua': {'Haugenstua'},
 'Haugenstuveien': {'Haugenstuveien'},
 'Hornfjellveien': {'Hornfjellveien'},
 'Høybråten': {'Høybråten'},
 'Høybråtenstien': {'Høybråtenstien'},
 'Høybråtenveien': {'Høybråtenveien'},
 'Idrettsveien': {'Idrettsveien'},
 'Kleiva': {'Kleiva'},
 'Kringveien': {'Kringveien'},
 'Langlibakken': {'Langlibakken'},
 'Linjebakken': {'Linjebakken'},
 'Linjeveien': {'Linjeveien'},
 'Lundåsveien': {'Lundåsveien'},
 'Lyngtrekket': {'Lyngtrekket'},
 'Myr

In [33]:
SCHEMA = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

In [51]:
"""
After auditing is complete the next step is to prepare the data to be inserted into a SQL database.
To do so you will parse the elements in the OSM XML file, transforming them from document format to
tabular format, thus making it possible to write to .csv files.  These csv files can then easily be
imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

We've already provided the code needed to load the data, perform iterative parsing and write the
output to csv files. Your task is to complete the shape_element function that will transform each
element into the correct format. To make this process easier we've already defined a schema (see
the schema.py file in the last code tab) for the .csv files and the eventual tables. Using the 
cerberus library we can validate the output against this schema to ensure it is correct.

## Shape Element Function
The function should take as input an iterparse Element object and return a dictionary.

### If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

The final return value for a "node" element should look something like:

{'node': {'id': 757860928,
          'user': 'uboot',
          'uid': 26299,
       'version': '2',
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
      'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

### If the element top level tag is "way":
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

The final return value for a "way" element should look something like:

{'way': {'id': 209809850,
         'user': 'chicago-buildings',
         'uid': 674454,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,
               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}
"""

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

OSM_PATH = "example.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

DEFAULT_TAG_TYPE = 'regular'

def gather_attribs(el, attr_schema):
    res = {}
    for k, v in attr_schema.items():
        if k in el.attrib:
            val = el.attrib[k]
            if 'coerce' in v:
                res[k] = (v['coerce'])(val)
            else:
                res[k] = val
    return res


def gather_tags(el, parent_attr):
    parent_id = parent_attr['id']
    for tag in el.iter("tag"):
        if 'k' not in tag.attrib or 'v' not in tag.attrib:
            continue
        k = tag.attrib['k']
        if PROBLEMCHARS.search(k) is not None:
            continue
        parts = k.split(':', 1)
        if len(parts) > 1:
            tag_type = parts[0]
            tag_key = parts[1]
        else:
            tag_type = DEFAULT_TAG_TYPE
            tag_key = k
        yield {'id': parent_id, 
               'key': tag_key, 
               'type': tag_type, 
               'value': tag.attrib['v']}


def gather_way_nodes(el, parent_attr):
    parent_id = parent_attr['id']
    position = 0
    for tag in el.iter('nd'):
        if 'ref' not in tag.attrib:
            continue
        try:
            yield {'id': parent_id, 
                   'node_id': int(tag.attrib['ref']), 
                   'position': position}
            position += 1
        except ValueError:
            # skip node with invalid reference
            pass
            
def shape_element(element, 
                  node_attr_fields=NODE_FIELDS, 
                  way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, 
                  default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    
    if element.tag == 'node':
        node_attribs = gather_attribs(element, SCHEMA['node']['schema'])
        tags = list(gather_tags(element, node_attribs))
        return {'node': node_attribs, 
                'node_tags': tags}
    
    elif element.tag == 'way':
        way_attribs = gather_attribs(element, SCHEMA['way']['schema'])
        way_nodes = list(gather_way_nodes(element, way_attribs))
        tags = list(gather_tags(element, way_attribs))
        return {'way': way_attribs, 
                'way_nodes': way_nodes, 
                'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(iter(validator.errors.items()))
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: v for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


process_map(DATA_PATH, validate=True)